## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [1]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/env")
    from google.colab import drive

    drive.mount("/content/drive")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /usr
Audited 1 package in 218ms
In Colab: True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool.

In [2]:
from ddgs import DDGS
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))


## Define your agent, task and crew

Note that the agent is provided with memory = True This provides the agent with conversation history. By activating memory=True in the crew, we make sure that the agents persists across multiple crew session calls, with different input states.

In [6]:
from crewai import Agent, Task, Crew, Process
from pydantic import BaseModel

class SqlQuery(BaseModel):
  sql_query: str
  dataframe: str

# Create a conversational agent with a friendly role/goal
chat_agent = Agent(
    role="Helpful Assistant",
    goal="Have a natural multi-turn conversation",
    backstory="You are a friendly assistant that remembers context and asks for clarification when needed.",
    memory=True  # enable memory for conversational context.
)

# Define a simple Task that represents a single AI response
task = Task(
    description="Respond appropriately to user's message, maintaining context. {input}",
    expected_output="A conversational reply",
    output_pydantic=SqlQuery,
    agent=chat_agent,
    tools=[web_search]
)

crew = Crew(
    agents=[chat_agent],
    tasks=[task],
    memory=True,  # ensure conversation state persists across turns
)

## Run the crew in a conversation loop
Since memory has been set to True, the crew will get access to the previous round of conversations, exibiting a conversational behaviour

In [ ]:
while user_input := input("You: ").strip():
    print("\n\n===========\nYou: ",user_input)
    result = crew.kickoff(inputs={"input": user_input})
    print("\nAssistant:", result)

You: Wrtie a query to find sec filling  of APPLE in wrds datasets


You:  Wrtie a query to find sec filling  of APPLE in wrds datasets

Assistant: I can help you with that! To find SEC filings for Apple Inc. using WRDS datasets, you would typically want to access the appropriate SEC filings database offered by WRDS. Here's how you might structure your query:

```
SELECT *
FROM sec.sec_data
WHERE cik = '0000320193'  -- CIK for Apple Inc.
AND filing_date >= '2020-01-01'
AND filing_date <= '2023-12-31'
```

In this SQL query:
- The `sec.sec_data` is a hypothetical reference to the SEC filings dataset in WRDS.
- You need to replace '0000320193' with the CIK (Central Index Key) for Apple Inc. which is specific to AAPL.
- You can adjust the date range as necessary.

If you need a practical example using the WRDS Python API, it might look something like this:

```python
import wrds

# Connect to WRDS
db = wrds.Connection()

# Query SEC filings for Apple Inc.
filings = db.raw_sql("""
    SELEC